<a href="https://colab.research.google.com/github/Adrok24/tp_digital_house/blob/main/NN_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns




In [ ]:
df = pd.read_csv('Telco.csv')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


Limpio los valores NaN y elimino la columna con código de cliente.

In [ ]:
df.drop('customerID', axis=1, inplace=True)
df['TotalCharges'] = df['TotalCharges'].replace(" ",np.nan)
df.dropna(how='any', inplace= True)
df['TotalCharges'] = df['TotalCharges'].astype(float)
df['SeniorCitizen'] = df['SeniorCitizen'].astype(int)



In [ ]:
columns_to_convert = ['Partner', 'Dependents','PhoneService','OnlineSecurity' ,
                      'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies','PaperlessBilling',
                      'Churn']


df[columns_to_convert] = df[columns_to_convert].replace(dict(Yes=1, No=0))


**Transformo las columnas con valores categóricos a valores numéricos.**

In [ ]:
categorical_columns = ['gender', 'MultipleLines', 'InternetService', 'Contract', 'PaymentMethod']

df = pd.get_dummies(data=df, columns= categorical_columns)

df.isnull().values.any()
df.isnull().sum().sum()

0

**Divido el dataset en datos de train/test 70/30**


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
X = pd.get_dummies(df.drop('Churn', axis=1), drop_first=True)
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=41)



Estandarizo las columnas numéricas y luego las adhiero de nuevo al dataset.

In [ ]:
from sklearn.preprocessing import StandardScaler


numerical_columns = ['tenure', 'MonthlyCharges', 'TotalCharges']

ss = StandardScaler()
X_train[numerical_columns] = ss.fit_transform(X_train[numerical_columns])
X_test[numerical_columns] = ss.fit_transform(X_test[numerical_columns])
#scl = pd.DataFrame(scl, columns=numerical_columns)

X_train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4922 entries, 3404 to 1989
Data columns (total 35 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   SeniorCitizen                            4922 non-null   int64  
 1   Partner                                  4922 non-null   int64  
 2   Dependents                               4922 non-null   int64  
 3   tenure                                   4922 non-null   float64
 4   PhoneService                             4922 non-null   int64  
 5   PaperlessBilling                         4922 non-null   int64  
 6   MonthlyCharges                           4922 non-null   float64
 7   TotalCharges                             4922 non-null   float64
 8   gender_Female                            4922 non-null   uint8  
 9   gender_Male                              4922 non-null   uint8  
 10  MultipleLines_No                         4922

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [ ]:
from imblearn.over_sampling import SMOTE
np.where(np.isnan(X_train)) 
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)





/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_train.shape[1]


35

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
import keras





In [ ]:
from tensorflow.keras import regularizers


def build_model(input_dim=(35,), layers=[64,64,1], optimizer='rmsprop', dropout_rate=0.2):
    # Instanciamos la clase del modelo secuencial
    model = Sequential()
    # Aplanamos los datos de entrada, sabemos que vamos a recibir imágenes
    model.add(Flatten(input_shape=input_dim))
    # Agregamos el resto de las capas con activación ReLU con excepción de la última
    for l in layers[:-1]:
        model.add(Dense(units=l, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
        model.add(Dropout(dropout_rate))
    # Agregamos la última capa con activación softmax
    model.add(Dense(units=layers[-1], activation='sigmoid'))
    # Compilamos el modelo con el optimizador seleccionado
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    # Retornamos el modelo compilado
    return model

In [ ]:
from keras import optimizers
import tensorflow as tf
layers = [[512, 1],
          [64, 32, 1],
          [12, 8, 1],
          [32, 32, 1]
         ]

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
      0.001,
      decay_steps=(X_train.shape[0]/32)*50,
      decay_rate=1,
      staircase=False)

optimizers = [optimizers.Adam(learning_rate = lr_schedule), optimizers.SGD(momentum=0.9, nesterov=True), optimizers.RMSprop()]

dropout_rates = [0.2, 0.5]

In [ ]:
import itertools
combinaciones = list(itertools.product(layers, optimizers, dropout_rates))
combinaciones

[([512, 1],
  0.2),
 ([512, 1],
  0.5),
 ([512, 1],
  0.2),
 ([512, 1],
  0.5),
 ([512, 1],
  0.2),
 ([512, 1],
  0.5),
 ([64, 32, 1],
  0.2),
 ([64, 32, 1],
  0.5),
 ([64, 32, 1],
  0.2),
 ([64, 32, 1],
  0.5),
 ([64, 32, 1],
  0.2),
 ([64, 32, 1],
  0.5)]

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

n_splits = 3
batch_size = 32
epochs = 50
verbose = 1

# Instanciamos los objetos early_stopping y reduce_lr y definimos una lista de callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=0)
early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=20, restore_best_weights=True, verbose=0)
callbacks_list = [early_stopping]


callbacks_list_param = callbacks_list

In [ ]:
global_history = []

In [ ]:
from sklearn import metrics

def make_prediction(X_test, y_test):

    yprednn=model.predict(X_test)
    yprednn=yprednn.round()
    print('Neural Network:\n {}\n'.format(
        metrics.classification_report(yprednn, y_test)))
    print('Neural Network:\n {}\n'.format(
        metrics.confusion_matrix(yprednn, y_test)))

In [ ]:
# Importamos KFold para hacer cross-validation
from sklearn.model_selection import KFold
import time

# Instanciamos el objeto KFold
kfold = KFold(n_splits=n_splits, shuffle=False)

# Recorremos las combinaciones y generamos distintos modelos a ensayar
for (layers, optimizer, dropout_rate) in combinaciones:
    print('\n\nEnsayando modelo con estructura {} y optimizador {}'.format(layers, optimizer, dropout_rate))
    
    # Construimos el modelo
    model = build_model(layers=layers, optimizer=optimizer, dropout_rate=dropout_rate)
    
    # Guardamos los pesos iniciales para usarlos en cada fold
    model.save_weights('initial_weights.h5')
    
    # Generamos los sets de train y val para ensayar el modelo
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train)):
        
        # Reiniciamos los pesos del modelo
        model.load_weights('initial_weights.h5')
        tic = time.time()
        # Lo entrenamos con el split de x_train e y_train correspondiente
        history = model.fit(x=X_train[train_idx],
                            y=y_train[train_idx],
                            batch_size=batch_size,
                            epochs=epochs,
                            validation_data=(X_train[val_idx], y_train[val_idx]),
                            callbacks=callbacks_list_param,
                            verbose=verbose
                           )
        toc=time.time()
        n_epochs = len(history.history['loss'])
        flod_timecost = round(toc-tic, 2)

        # Evaluamos en train y en val (estos mismos valores los podemos sacar de history)
        train_loss, train_acc = model.evaluate(X_train[train_idx], y_train[train_idx])
        val_loss, val_acc = model.evaluate(X_train[val_idx], y_train[val_idx])
        yprednn=model.predict(X_test)
        yprednn=yprednn.round()
        
        # Agregamos esta corrida a la historia global
        global_history.append({'fold':fold, 
                               'layers':','.join([str(elem) for elem in layers]), 
                               'optimizer':str(type(optimizer)),
                               'dropout':dropout_rate,
                               'batch_size': batch_size,
                               'stoped_at_epoch': n_epochs,
                               'train_loss':train_loss,
                               'train_acc':train_acc,
                               'val_loss':val_loss,
                               'val_acc':val_acc,
                               'time': flod_timecost,
                               'history':history,
                               'prediction':yprednn
                              })



Ensayando modelo con estructura [512, 1] y optimizador <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7fd2fa9e8940>
Epoch 1/50
152/152 [==============================] - 1s 3ms/step - loss: 0.5094 - accuracy: 0.7943 - val_loss: 0.5603 - val_accuracy: 0.7465
Epoch 2/50
152/152 [==============================] - 0s 3ms/step - loss: 0.4724 - accuracy: 0.8081 - val_loss: 0.5766 - val_accuracy: 0.7254
Epoch 3/50
152/152 [==============================] - 0s 3ms/step - loss: 0.4627 - accuracy: 0.8052 - val_loss: 0.5561 - val_accuracy: 0.7353
Epoch 4/50
152/152 [==============================] - 0s 3ms/step - loss: 0.4524 - accuracy: 0.8085 - val_loss: 0.6730 - val_accuracy: 0.6621
Epoch 5/50
152/152 [==============================] - 0s 3ms/step - loss: 0.4512 - accuracy: 0.8100 - val_loss: 0.5734 - val_accuracy: 0.7113
Epoch 6/50
152/152 [==============================] - 0s 3ms/step - loss: 0.4426 - accuracy: 0.8170 - val_loss: 0.5746 - val_accuracy: 0.7093
Epoch 7/50
152/15

In [ ]:
import pandas as pd
df = pd.DataFrame(global_history)
df.head()


In [ ]:
def plot_history(name, history, legend, plot_val=True):
    fig, ax = plt.subplots(1,2,figsize=(14,6))
    fig.suptitle(name)
    
    if not isinstance(history, list):
        history = [history]
        
    for h in history:
        acc = h.history['accuracy']
        loss = h.history['loss']
        if plot_val:
            val_loss = h.history['val_loss']
            val_acc = h.history['val_accuracy']
        epochs = range(1, len(acc) + 1)

        ax[0].set_title('Loss')
        ax[0].set_xticks(ticks=epochs)
        ax[0].set_ylabel('Loss')
        
        ax[0].plot(epochs, loss)
        if plot_val:
            ax[0].plot(epochs, val_loss)
            
        ax[1].set_title('Accuracy')
        ax[1].set_xticks(ticks=list(epochs))
        ax[1].set_xlabel('Epochs')
        ax[1].set_ylabel('Accuracy')
        ax[1].plot(epochs, acc)
        if plot_val:
            ax[1].plot(epochs, val_acc)
        
    ax[0].legend([l+' loss' for l in legend])
    ax[1].legend([l+' accuracy' for l in legend])

from sklearn import metrics

def make_prediction(X_test, y_test):

    yprednn=model.predict(X_test)
    yprednn=yprednn.round()
    print('Neural Network:\n {}\n'.format(
        metrics.classification_report(yprednn, y_test)))
    print('Neural Network:\n {}\n'.format(
        metrics.confusion_matrix(yprednn, y_test)))
        

In [ ]:
for history in global_history:
    plot_history(history['layers'], history['history'], legend=['train', 'val'])

In [ ]:
for history in global_history:
    print(history['layers'], history['optimizer'])
    print('Neural Network:\n {}\n'.format(
        metrics.classification_report(history['prediction'], y_test)))
    print('Neural Network:\n {}\n'.format(
        metrics.confusion_matrix(history['prediction'], y_test)))

In [ ]:
import pandas as pd
df_history = pd.DataFrame(global_history)
df_history.sort_values(by='val_acc', ascending=False,inplace=True)

df_history.groupby(['layers','optimizer', 'dropout', 'batch_size']).mean().sort_values(by='val_acc', ascending=False).drop(['fold'],axis=1)

# Nos quedamos con las redes [64, 32, 1] y [512,1] ambas con RMSprop y 0.2 de Drop Out, y las probamos para más epochs.

In [ ]:
model = Sequential()
model.add(Dense(X_train.shape[1]))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

optimizer = keras.optimizers.RMSprop()
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    validation_split=0.3,
                    batch_size=32, epochs=500)



In [ ]:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12,8))
plt.title('Modelo de base')
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.xticks(ticks=epochs)
plt.ylabel('Loss')
plt.legend(['Training loss', 'Validation loss'])

plt.figure(figsize=(12,8))
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.xticks(ticks=list(epochs))
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training accuracy', 'Validation accuracy']);


In [ ]:
yprednn=model.predict(X_test)
yprednn=yprednn.round()
print('Neural Network:\n {}\n'.format(
    metrics.classification_report(yprednn, y_test)))
print('Neural Network:\n {}\n'.format(
    metrics.confusion_matrix(yprednn, y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
cm = confusion_matrix(y_true=y_test, y_pred=yprednn)


def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
plot_confusion_matrix(cm=cm, classes=['Yes', 'No'], title='Confusion Matrix')

# # Ahora probamos con la red de [512, 1]

In [ ]:
model = Sequential()
model.add(Dense(X_train.shape[1]))
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

optimizer = keras.optimizers.RMSprop()
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    validation_split=0.3,
                    batch_size=32, epochs=500)


In [ ]:
yprednn=model.predict(X_test)
yprednn=yprednn.round()
print('Neural Network:\n {}\n'.format(
    metrics.classification_report(yprednn, y_test)))

In [ ]:
cm = confusion_matrix(y_true=y_test, y_pred=yprednn)
plot_confusion_matrix(cm=cm, classes=['Yes', 'No'], title='Confusion Matrix')

# # Vemos que performa mejor la red con una estructura [64,32,1]